In [ ]:
# Required pkg
import Pkg;
Pkg.add("Images")
Pkg.add("LinearAlgebra")
Pkg.add("Plots")
Pkg.add("Interpolations")
Pkg.add("DataFrames")
Pkg.add("XLSX")
using Images, Statistics, LinearAlgebra, Plots, Interpolations, Random, DataFrames, XLSX

### [Auxiliary functions] to get: training_images, testing_images
#### Each with a size = M * N^2

M = Number of images (min_images_per_folder)

N^2 = A vector of the original image_size*image_size image

In [ ]:
# Get useful folders

# Return a Vector{String} with all the folders with {min_images_per_folder} inside.
function get_useful_folders(min_images)
    ### Getting images
    useful_folders = Vector{String}()
    
    itr = walkdir(base_dir)    
    (root, dirs, files) = first(itr)
    for d in dirs
        images_on_folder = readdir(base_dir * d)
        if size(images_on_folder)[1] >= min_images
            push!(useful_folders, base_dir * d * "/")
        end
    end
    useful_folders
end

# After filter by get_useful_folders(), get random {n_folders}
function get_random_n_folders(min_images_per_folder)
    # Get all the useful folders
    useful_folders = get_useful_folders(min_images_per_folder)
    # Get random folders from {useful_folders}
    rand(useful_folders, n_folders)
end

# Get all images from {n_folders} random folders, each case for {training_images} and {testing_images}
# Return a Matrix, size = M * N^2
#     M = Number of images (min_images_per_folder)
#     N^2 = A row vector of the original N*N image (now: 1*N^2)

function get_images(float_type, working_with=nothing)
    
    # Pick {n_folders} to work with if {working_with} not provided
    if working_with === nothing
       working_with = get_random_n_folders(min_images_per_folder)
    end    
        
    training_images = zeros(float_type, size(working_with, 1)*n_training_images,  image_size*image_size)
    testing_images = zeros(float_type, size(working_with, 1)*n_testing_images,  image_size*image_size)
    
    # For every folder, get pictures
    for (i, folder) in enumerate(working_with)
        cdir = readdir(folder)        
        
        # Google Drive create a "desktop.ini", then I have to "findfirst" and delete element
        splice!(cdir, findfirst(contains("desktop.ini"), cdir))
            
        training_count = 1
        testing_count = 1

        for image_relative_path in cdir 
            
            row_im = vec(float_type.(Gray.( imresize(load(folder * image_relative_path), image_size, image_size) )))' # 1 x N*N
            #display(Gray.(load(folder * image_relative_path)))
            
            if training_count <= n_training_images
                training_images[ (n_training_images*(i-1) + training_count),:] = row_im
                training_count += 1
            elseif testing_count <= n_testing_images
                testing_images[ (n_testing_images*(i-1) + testing_count),:] = row_im
                testing_count += 1
            else
                break  # Stop here if both training and testing images are collected
            end
        end
    end
    (training_images, testing_images)
end

In [ ]:
function show_training_images(images)
    for i in 1:n_folders
        row = vcat([ Gray.(reshape(images[j,:], image_size, image_size)) for j in (n_training_images*(i-1))+1:((n_training_images*(i-1))+n_training_images)])
        display(row)
    end    
end

function show_testing_images(images)
    for i in 1:n_folders
        row = vcat([ Gray.(reshape(images[j,:], image_size, image_size)) for j in (n_testing_images*(i-1))+1:((n_testing_images*(i-1))+n_testing_images)])
        display(row)
    end    
end

# Getting the basics

### μ, A, C, eigenvalues, eigenvectors

#### n_folders
Number of folders to use, randomly selected. **Between 1 and _size(get_useful_folders(n_folders), 1)_**

#### n_training_images
Number of training images to pick from every folder

#### n_testing_images
Same but for testing

#### min_images_per_folder
Before, it will filter the folders on the directory by {min_images_per_folder}

In [ ]:
# ./lfw/  = Color, not aligned
# ./lfw2/ = Black and white, already aligned (eyes)
base_dir = "./lfw/"
image_size = 64

n_folders = 10
n_training_images = 12
n_testing_images = 6

min_images_per_folder = n_training_images + n_testing_images + 1; # +1 because google drive creating desktop.ini

#### (training_images, testing_images), μ, A, eigenvalues, eigenvectors, elapsed_time, mem_used

In [ ]:
function setup_basics(float_type, folders=nothing)
    (training_images, testing_images) = get_images(float_type, folders)
    
    # Mean
    μ = mean(training_images, dims = 1)

    # Centered matrix
    A = training_images.-μ

    # Covariance (N2xN2) and eigen
    run_block = @timed begin
        (eigenvalues, eigenvectors) = eigen(cov(A), sortby = x -> -x)
    end
    
    (eigenvalues, eigenvectors) = run_block.value # Again
    elapsed_time = run_block.time
    mem_used = run_block.bytes / 1024^3 # Bytes to Gigabytes
    
    (training_images, testing_images), μ, A, eigenvalues, eigenvectors, elapsed_time, mem_used
end

## Set folders to use, or random

In [ ]:
# Set folders to use, or random

# Get random folders 
#random_n_folders = get_random_n_folders(min_images_per_folder)

# Fixed
random_n_folders = [
 "./lfw/Guillermo_Coria/",
 "./lfw/Atal_Bihari_Vajpayee/",
 "./lfw/Kofi_Annan/",
 "./lfw/Nicole_Kidman/",
 "./lfw/Kofi_Annan/",
 "./lfw/Pervez_Musharraf/",
 "./lfw/Richard_Myers/",
 "./lfw/Tom_Daschle/",
 "./lfw/Amelie_Mauresmo/",
 "./lfw/Atal_Bihari_Vajpayee/"
];

# Reconstruction
## Measuring _data_ differences for: Float 16, 32, 64, for every k on every image

In [ ]:
precisions = [Float16, Float32, Float64]

#### Setup (this block is repeated several times below for simplicity, but it's the same)

In [1]:
image_size = 64

n_folders = 16
n_training_images = 32
n_testing_images = 4

min_images_per_folder = n_training_images + n_testing_images + 1; # +1 because google drive creating desktop.ini

In [2]:
# Check if there are as many folder after the filter using: min_images_per_folder
println("We want $n_folders folders with $(min_images_per_folder-1) min_images_per_folder, we have: $(size(get_useful_folders(min_images_per_folder), 1))")

LoadError: UndefVarError: `get_useful_folders` not defined

#### [Auxiliary functions]:

In [ ]:
#
# Returns a VECTOR with the norm error between original and reconstructed using k vectors.
#
#           |  k=1    |    k=2   |   ...   |  k = index_X_percent |
#-----------------------------------------------------------------|
#   Image   |  error  |   error  |  ...    |        error         |

function reconstruct_for_k_eigenfaces(U, A, μ, training_images, display_images)
    # Projections
    P = transpose(U) * transpose(A)

    # Reconstruction using k eigenvectors
    reconstructed_images = (P' * U').+μ
    
    if display_images == true
        display(
            [
                Gray.(reshape(training_images[2,:], image_size, image_size)), # Original
                Gray.(reshape(reconstructed_images[2,:], image_size, image_size))  # Using k eigenfaces
            ]
        )
    end
    return norm.(eachrow(training_images - reconstructed_images), 2) # row_norms
end


# For every precision creates a table with the different reconstruction errors for every k on every image.
# Example:

#           |  k=1    |    k=2   |   ...   |  k = index_X_percent |
#-----------------------------------------------------------------|
# Image_1   |  error  |   error  |  ...    |        error         |
#-----------------------------------------------------------------|
# Image_2   |  error  |   error  |  ...    |        error         |
#-----------------------------------------------------------------|
#   ...     |   ...   |    ...   |   ...   |        ...           |
#-----------------------------------------------------------------|
# Image_M   |  error  |   error  |  ...    |        error         |
#-----------------------------------------------------------------|


function reconstruc_all_for_X_percent_of_k(percent)
    errors_by_precision = []
    
    # Random
    training_set = get_random_n_folders(min_images_per_folder)
    
    for used_precision in precisions
        
        # Getting the basics
        ((training_images, testing_images), μ, A, eigenvalues, eigenvectors, elapsed_time, mem_used) = setup_basics(used_precision, training_set);
        
        # Using the to XX% most significant of all eigenvalues
        cumulative_eigenvalues = cumsum(eigenvalues./sum(eigenvalues))
        index_X_percent = findfirst(cumulative_eigenvalues .>= (percent/100))
        
        errors_by_k = zeros(Float64, n_folders*n_training_images, index_X_percent)
        
        display_data = false
        for k in 1:index_X_percent
            
            if k == index_X_percent
                display_data = true
            end
            errors_by_k[:,k] = reconstruct_for_k_eigenfaces(eigenvectors[:, 1:k], A, μ, training_images, display_data)
        end
            
        push!(errors_by_precision, errors_by_k)
    end
    errors_by_precision
end



#### Run function

In [ ]:
errors_by_precision = reconstruc_all_for_X_percent_of_k(99);

In [ ]:
xlsx_file = XLSX.writetable("temp_table.xlsx", DataFrame(errors_by_precision[3], :auto),  overwrite=true, sheetname="Float")


In [ ]:
mean( vcat(errors_by_precision[3][:,end], errors_by_precision[2][:,end], errors_by_precision[1][:,end]) )

In [ ]:
println("Mean Float16: $(mean(errors_by_precision[1][:,end]))")
println("Mean Float32: $(mean(errors_by_precision[2][:,end]))")
println("Mean Float64: $(mean(errors_by_precision[3][:,end]))")


In [ ]:
errors_by_precision[3]

In [ ]:
errors_by_precision[2]

In [ ]:
errors_by_precision[1]

#### Analysis

In [ ]:
# Working with eps 64
epsilon = eps(Float16)

# Absolite diff bigger than eps? (64 vs 16)
differences = errors_by_precision[3] - errors_by_precision[1]
result = abs.(differences) .> epsilon

# Results
println("Differences: ")
println(differences)
println("Results: ")
println(result)

In [ ]:
result

## Measuring _mem and time_ differences for: Float 16, 32, 64 on setup

#### Run block

In [ ]:
array_n_folders[end]

In [ ]:
#
# Memory and time used to calculate covariance matrix and eigen for F16, 32, 64
#
mem_time_by_precision = []
    
#
# Original values -> Aux
aux_n_folders = n_folders
aux_n_training_images = n_training_images
aux_n_testing_images = n_testing_images
aux_random_n_folders = random_n_folders

# Different values to test:
array_n_folders = [4, 8] # <-------- Modify this
array_n_training_images = [8, 16] # <-------- Modify this
n_testing_images = 4

# Update {min_images_per_folder} using the bigger values
min_images_per_folder = n_testing_images + array_n_training_images[end] + 1; # +1 because google drive creating desktop.ini

# Use the new {min_images_per_folder} to get the folders
training_set = get_random_n_folders(min_images_per_folder)

for used_precision in precisions

    data = zeros(Float64, size(array_n_folders,1)*size(array_n_training_images,1), 4) 
    index = 1
    # Table data:

    # n_folders | n_training_images | elapsed_time | mem_used
    #    ..              ..                ..           ..
    #    ..              ..                ..           ..

    for aux1 in array_n_folders
        n_folders = aux1

        for aux2 in array_n_training_images
            n_training_images = aux2

            # Working here...
            ((training_images, testing_images), μ, A, eigenvalues, eigenvectors, elapsed_time, mem_used) = setup_basics(used_precision, training_set[1:n_folders]);

            # Save data:
            data[index,:] = [n_folders, n_training_images, elapsed_time, mem_used]
            index += 1            
        end
    end

    push!(mem_time_by_precision, data)
end

# Aux -> Original values
n_folders = aux_n_folders
n_training_images = aux_n_training_images
n_testing_images = aux_n_testing_images;

In [ ]:
[]

#### Show data

In [ ]:
# Encabezado de la tabla
println("Float16:")
println("n_folders | n_training_images | elapsed_time | mem_used")
mem_time_by_precision[1]

In [ ]:
println("Float32:")
mem_time_by_precision[2]

In [ ]:
println("Float64:")
mem_time_by_precision[3]

## Plot eigenvalues

#### Setup data first:

In [ ]:
used_precision = Float32

image_size = 128

n_folders = 2
n_training_images = 4
n_testing_images = 4

min_images_per_folder = n_training_images + n_testing_images + 1; # +1 because google drive creating desktop.ini

((training_images, testing_images), μ, A, eigenvalues, eigenvectors, elapsed_time, mem_used) = setup_basics(used_precision);

X_percent = 99
X_percent = X_percent / 100

#### Run block:

In [ ]:
cumulative_eigenvalues = cumsum(eigenvalues ./ sum(eigenvalues))
index_X_percent = findfirst(cumulative_eigenvalues .>= X_percent)

# To put X_percent in the middle
x_range = 1:index_X_percent * 2

# Plot 2 curves con el rango x limitado
plot(
    [eigenvalues[x_range] ./ sum(eigenvalues)], 
    label="eigenvalues",
    title="$(X_percent)% coverage at the $index_X_percent-th elem of $(size(eigenvalues, 1)) (~$(round(index_X_percent/(size(eigenvalues, 1)), digits=4))%)",
    legend=:outerbottom,
    titlefontsize=12
)

# Plot cumulative eigenvalues con el rango x limitado
plot!(cumulative_eigenvalues[x_range], label="cumulative eigenvalues")

# Add X% text
annotate!([(index_X_percent, cumulative_eigenvalues[index_X_percent], text("$(X_percent)%", 8))])

# Add x line
vline!([index_X_percent], line=:black, linewidth=0.5, label=nothing)

# Add y line
hline!([cumulative_eigenvalues[index_X_percent]], line=:black, linewidth=0.5, label=nothing)

In [ ]:
savefig("2x8.png")

# Recognition

In [ ]:
#show_training_images(training_images)

In [ ]:
#show_testing_images(testing_images)

#### Setup data first:

In [ ]:
image_size = 64

n_folders = 16
n_training_images = 32
n_testing_images = 8

min_images_per_folder = n_training_images + n_testing_images + 1; # +1 because google drive creating desktop.ini

# Selecting the top {X_percent}% of the eigenvalues
X_percent = 99
X_percent = X_percent / 100

In [ ]:
# Check if there are as many folder after the filter using: min_images_per_folder
println("We want $n_folders folders with $(min_images_per_folder-1) min_images_per_folder, we have: $(size(get_useful_folders(min_images_per_folder), 1))")

#### Aux functions:

In [ ]:
# Use k components to test accuracy.

function test_for_k_components(A, eigenfaces, random_n_folders, testing_images, display_data)
    P = transpose(eigenfaces)*transpose(A);
    
    if display_data == true
        println("Testing for: $(typeof(A)) and $(size(eigenfaces, 2)) eigenfaces")
    end

    correct = 0.0
    all_min_dif = [] # We collect all the min norm diff for every k
    all_variance = [] # The variance between all images
    
    for image_test_index in 1:size(testing_images,1)
        x_i = testing_images[image_test_index,:]' - μ
        P_i = transpose(eigenfaces) * transpose(x_i);

        dist=[]
        for j in 1:size(P,2)
            push!(dist, norm(P[:,j] - P_i))
        end
        pred = argmin(dist)
            
        push!(all_min_dif, dist[pred])
        push!(all_variance, var(dist))

        celeb_tested = random_n_folders[ div(image_test_index - 1, n_testing_images) + 1 ]            
        celeb_predicted = random_n_folders[ div(pred - 1, n_training_images) + 1 ]
        
        if (celeb_tested == celeb_predicted)
            correct+=1
        end   
                
        if display_data == true
            println("_________________________________")
            println("Current: $celeb_tested (#$image_test_index)")
            println("Pred:    $celeb_predicted (#$pred)")

            println("Correct: $(correct/image_test_index*100)% ")

            IJulia.display([
                Gray.(reshape(testing_images[image_test_index,:], image_size, image_size)),
                Gray.(reshape(training_images[pred,:], image_size, image_size))
            ])
        end
    end
            
    accuracy = (correct/size(testing_images,1))*100
    (accuracy, vec(all_min_dif), vec(all_variance))
end


#### Run block:

In [ ]:
# The training set (USE THE SAME FOR EVERY PRECISION)
training_set = get_random_n_folders(min_images_per_folder)

# Firts run, to get eigenvalues and then -> index_X_percent
used_precision = Float16
((training_images, testing_images), μ, A, eigenvalues, eigenvectors, elapsed_time, mem_used) = setup_basics(used_precision, training_set);

# Testing for the {X_percent}%
cumulative_eigenvalues = cumsum(eigenvalues ./ sum(eigenvalues))
index_X_percent = findfirst(cumulative_eigenvalues .>= X_percent)


# Table format for {accuracy_by_precision}
#
#           |  Float16  |  Float32  |  Float64  |
#-----------|-----------|-----------|-----------|
#    k=1    | correct%  | correct%  | correct%  |
#    k=2    | correct%  | correct%  | correct%  |
#    ...    |   ...     |   ...     |   ...     |
#   k = X%  | correct%  | correct%  | correct%  |


# Table format for {recognition_dif_by_precision}
#
# We save the minimum difference (norm) for every test image, for each k (1 to 99% eigenface space)
#
#                |  k=1    |    k=2   |   ...   |  k = index_X_percent |
#----------------------------------------------------------------------|
# Test_image_1   |  error  |   error  |  ...    |        error         |
#----------------------------------------------------------------------|
# Test_image_2   |  error  |   error  |  ...    |        error         |
#----------------------------------------------------------------------|
#        ...     |   ...   |    ...   |   ...   |        ...           |
#----------------------------------------------------------------------|
# Test_image_M   |  error  |   error  |  ...    |        error         |
#-----------------------------------------------------------------     |

# Table format for {recognition_var_by_precision}
#
# The same, but for every test image on every k, we calculate de variance across all the matches, this is
# projected training image vs a single projected testing image
#
#                |  k=1      |    k=2    |   ...    |  k = index_X_percent |
#--------------------------------------------------------------------------|
# Test_image_1   | total_var | total_var |   ...    |      total_var       |
#--------------------------------------------------------------------------|
# Test_image_2   | total_var | total_var |   ...    |      total_var       |
#--------------------------------------------------------------------------|
#        ...     |    ...    |    ...    |   ...    |        ...           |
#--------------------------------------------------------------------------|
# Test_image_M   | total_var | total_var |   ...    |      total_var       |
#--------------------------------------------------------------------------|


# The data:
#
accuracy_by_precision = zeros(Float64, index_X_percent, 3)
recognition_dif_by_precision = []
recognition_var_by_precision = []

# TODO: Iterations?

for (col, used_precision) in enumerate(precisions)
    ((training_images, testing_images), μ, A, eigenvalues, eigenvectors, elapsed_time, mem_used) = setup_basics(used_precision, training_set);

    recognition_diff = zeros(Float64, n_folders * n_testing_images, index_X_percent)
    recognition_var = zeros(Float64, n_folders * n_testing_images, index_X_percent)
        
    for k in 1:index_X_percent

        # accuracy = A single value
        # diff     = A complete column
        (accuracy, diff, var) = test_for_k_components(A, eigenvectors[:,1:k], training_set, testing_images, false)

        accuracy_by_precision[k,col] = accuracy
        recognition_diff[:,k] = diff
        recognition_var[:,k] = var
    end
    
    push!(recognition_dif_by_precision, recognition_diff)
    push!(recognition_var_by_precision, recognition_var)
end

In [ ]:
accuracy_by_precision

In [ ]:
 plot(accuracy_by_precision[:,1], label = "Float16", title = "Comparativa de porcentaje de aciertos", xlabel = "Número de eigenvectores usados", ylabel = "% de aciertos")
plot!(accuracy_by_precision[:,2], label = "Float32")
plot!(accuracy_by_precision[:,3], label = "Float64")

In [ ]:
savefig("comparativa.png")

#### Minimum norm difference

In [ ]:
xlsx_file = XLSX.writetable("temp_table.xlsx", DataFrame(recognition_dif_by_precision[1], :auto), overwrite=true)

In [ ]:
recognition_dif_by_precision[3]

In [ ]:
recognition_dif_by_precision[2]

In [ ]:
recognition_dif_by_precision[1]

In [ ]:
[((recognition_dif_by_precision[2][:, i] - recognition_dif_by_precision[1][:, i])) for i in 1:size(recognition_dif_by_precision[1], 2)]

In [ ]:
eps(Float32)

In [ ]:
graph_diff = hcat([Int16(k) for k in 1:size(recognition_dif_by_precision[1], 2)] ,[mean((recognition_dif_by_precision[2][:, i] - recognition_dif_by_precision[1][:, i]) .> eps(Float32))*100 for i in 1:size(recognition_dif_by_precision[1], 2)])

In [ ]:
# Extraer columnas para x e y
x = graph_diff[:, 1]
y = graph_diff[:, 2]

# Crear el gráfico de dispersión
plot(x, y, marker=:circle, line=:auto, xlabel="Número de eigenvectores usados", ylabel="% de diferencias que superaron eps", titlefontsize=12, title = "Comparativa Float32 y Float16 entre normas de diferencias", legend=false)
# Establecer el rango del eje y de 0 a 100%
ylims!(0, 100)


In [ ]:
savefig("32vs16.png")

#### Minimum var

In [ ]:
recognition_var_by_precision[3]

In [ ]:
recognition_var_by_precision[2]

In [ ]:
recognition_var_by_precision[1]

In [ ]:
recognition_var_by_precision[3]-recognition_var_by_precision[1]

In [ ]:
graph_var = hcat([Int16(k) for k in 1:size(recognition_var_by_precision[1], 2)] ,[mean((recognition_var_by_precision[2][:, i] - recognition_var_by_precision[1][:, i]) .> eps(Float32))*100 for i in 1:size(recognition_dif_by_precision[1], 2)])

In [ ]:
# Extraer columnas para x e y
x = graph_var[:, 1]
y = graph_var[:, 2]

# Crear el gráfico de dispersión
plot(x, y, marker=:circle, line=:auto, xlabel="Número de eigenvectores usados", ylabel="% de diferencias que superaron eps", titlefontsize=12, title = "Comparativa Float32 y Float16 entre varianzas", legend=false)
# Establecer el rango del eje y de 0 a 100%
ylims!(0, 100)


In [ ]:
savefig("32vs16_var.png")

#### Comparison

In [ ]:
# Working with eps 64
    epsilon = eps(Float32)

# Abs diff bigger than eps?
# [3] = 64
# [2] = 32
# [1] = 16
result = abs.(recognition_dif_by_precision[3] - recognition_dif_by_precision[1]) .> epsilon

# Results
println("1 = Difference between values is > than epsilon $(typeof(epsilon))")
println("0 = No\n")
println(result)

In [ ]:
"""
DATA FRAMES ---> EXCEL

# Define DF
df = DataFrame(
    used_precision = UInt8[],
    image_size = UInt16[],
    n_folders = UInt16[],
    n_training_images = UInt8[],
    n_testing_images = UInt8[],
    eigen_elapsed_time = Float64[],
    used_eigenfaces = UInt8[],
    norm_error = Float64[],
    acurracy = Float16[]
)

# Define a row
row_dict = Dict{String, Any}()

# Save data
row_dict["used_precision"] = p
row_dict["image_size"] = image_size
row_dict["persons"] = used_persons
row_dict["images_db"] = images_to_db
row_dict["images_to_test"] = images_to_test
row_dict["covariance_calc_time"] = covariance_elapsed_time
row_dict["eigen_elapsed_time"] = eigen_elapsed_time
row_dict["used_eigenfaces"] = n_components
row_dict["norm_error"] = error
row_dict["acurracy"] = acurracy

# Push to DF
push!(df, row_dict)

# Write excel
XLSX.writetable("eigenframe.xlsx", df, overwrite=true, sheetname="Recognition")
""";